# Fused tiled matmul (large N)



In [6]:
import torch

M, N, N1, K = 15, 9, 12, 12

A = torch.rand((M, K))
B = torch.rand((K, N))
C = torch.rand((N, N1))

In [20]:
block_M, block_N, block_N1, block_K = M // 3, N // 3, N1 // 3, K // 3

output_ABC = torch.zeros((M, N1))
output_AB = torch.zeros((M, N))

total_read = 0
total_write = 0

for index_M in range(0, M, block_M):
    start_M = index_M
    end_M = index_M + block_M

    for index_N in range(0, N, block_N):
        start_N = index_N
        end_N = index_N + block_N
        accumulator0 = torch.zeros((block_M, block_N))

        for index_K in range(0, K, block_K):
            start_K = index_K
            end_K = index_K + block_K

            tile_A = A[start_M:end_M, start_K:end_K]
            total_read += tile_A.numel()
            tile_B = B[start_K:end_K, start_N:end_N]
            total_read += tile_B.numel()
            # @ means matmul in numpy and pytorch
            accumulator0 += tile_A @ tile_B
        output_AB[start_M:end_M, start_N:end_N] = accumulator0

        for index_N1 in range(0, N1, block_N1):
            start_N1 = index_N1
            end_N1 = index_N1 + block_N1
            tile_C = C[start_N:end_N, start_N1:end_N1]
            total_read += tile_C.numel()
            # do it as an atomic add
            output_ABC[start_M:end_M, start_N1:end_N1] += accumulator0 @ tile_C
            total_write += output_ABC[start_M:end_M, start_N1:end_N1].numel()

assert torch.allclose(output_AB, A @ B)
assert torch.allclose(output_ABC, (A @ B) @ C, atol=0.1)

In [22]:
total_write

540

In [23]:
total_read

1188